In [1]:
import sys
import unified_planning as up
import up_symk
import up_fast_downward
from unified_planning.shortcuts import *
from unified_planning.io import PDDLReader

In [2]:
# Read a simple PDDL gripper problem from file

reader = PDDLReader()
pddl_problem = reader.parse_problem("gripper-domain.pddl", "gripper-prob01.pddl")
pddl_problem.add_quality_metric(MinimizeSequentialPlanLength())

# Validator
pv = PlanValidator(problem_kind=pddl_problem.kind)

In [5]:
# Solve problem optimally with symk (bidirectional symbolic search)

with OneshotPlanner(name='symk-opt') as planner:
    result = planner.solve(pddl_problem) # output_stream=sys.stdout
    if result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
        pv_res = pv.validate(pddl_problem, result.plan)
        cost = pv_res.metric_evaluations[pddl_problem.quality_metrics[0]]
        print(f"{planner.name} found this plan: {result.plan} with cost {cost}.")
    else:
        print("No plan found.")

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 3 of `/tmp/ipykernel_17750/1651508481.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md)
  * Description: SymK is a state-of-the-art domain-independent classical optimal and top-k planner.

SymK (with optimality guarantee) found this plan: [pick(ball2, rooma, right), pick(ball1, rooma, left), move(rooma, roomb), drop(ball2, roomb, right), drop(ball1, roomb, left), move(roomb, rooma), pick(ball3, rooma, left), pick(ball4, rooma, right), move(rooma, roomb), drop(ball3, roomb, left), drop(ball4, roomb, right)] with cost 11.


In [6]:
# Find the best 10 with SymK (10 is currently hardcoded)

with AnytimePlanner(name='symk-opt', params={"number_of_plans": 5}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            pv_res = pv.validate(pddl_problem, result.plan)
            cost = pv_res.metric_evaluations[pddl_problem.quality_metrics[0]]
            print(f"Plan {i+1}: {result.plan} with cost {cost}.")
            print()
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print(f"{planner.name} found {i} optimal plans.")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.") 

  *** Credits ***
  * In operation mode `AnytimePlanner` at line 3 of `/tmp/ipykernel_17750/1499908851.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md)
  * Description: SymK is a state-of-the-art domain-independent classical optimal and top-k planner.

Plan 1: [pick(ball2, rooma, right), pick(ball1, rooma, left), move(rooma, roomb), drop(ball1, roomb, left), drop(ball2, roomb, right), move(roomb, rooma), pick(ball3, rooma, left), pick(ball4, rooma, right), move(rooma, roomb), drop(ball3, roomb, left), drop(ball4, roomb, right)] with cost 11.

Plan 2: [pick(ball2, rooma, right), pick(ball1, rooma, left), move(rooma, roomb), drop(ball1, roomb, left), drop(ball2, roomb, right), move(roomb, rooma), pick(ball3, rooma, left), pick(ball4, rooma, right), move(rooma, roomb), drop(ball4, roomb, right), drop(ball3, roomb, left)] with cost 11.

Plan 3: [pick(ball2, rooma, right), pick